In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_excel("./Datasets/HVAC_limpio.xlsx", "HVAC_limpio", index_col=0)
#data

In [3]:
data['Fecha- hora de lectura'] = (data['Fecha- hora de lectura'] - data['Fecha- hora de lectura'].min())  / np.timedelta64(1,'D')
#data.drop(['Fecha- hora de lectura'], axis=1, inplace=True)

In [4]:
data.columns

Index(['Fecha- hora de lectura', 'POTENCIA TRAFO 2', 'POTENCIA TRAFO 3',
       'POTENCIA TRAFO 4', 'POTENCIA TRAFO 5', 'POTENCIA MEDIA CONECTADA',
       'CONTROL FRÍO', 'POTENCIA BOMBA CALOR FELIPE',
       'POTENCIA BOMBA CALOR CARLOS',
       'TEMPERATURA AMBIENTE BOMBA CALOR CARLOS',
       'TEMPERATURA AMBIENTE BOMBA CALOR FELIPE', 'TEMPERATURA EXTERIOR',
       'CAPACIDAD GRUPO DE FRÍO 2', 'POTENCIA GRUPO FRÍO 1',
       'POTENCIA GRUPO FRÍO 2', 'ENTRADA AGUA A TORRE 1',
       'SALIDA AGUA TORRE 1', 'ENTRADA AGUA A TORRE 2', 'SALIDA AGUA TORRE 2',
       'C_O_P MÁQUINA GRUPO FRÍO 1', 'C_O_P MÁQUINA GRUPO FRÍO 2',
       'C_O_P BOMBA CALOR CARLOS', 'C_O_P BOMBA CALOR FELIPE',
       'TEMPERATURA SALIDA BOMBA CALOR CARLOS',
       'TEMPERATURA SALIDA BOMBA CALOR FELIPE',
       'KILO CALORÍAS GENERADAS BOMBA CALOR CARLOS',
       'KILO CALORÍAS GENERADAS BOMBA CALOR FELIPE',
       'KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 1',
       'KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2',
    

In [5]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim != None:
        plt.ylim(*ylim)
    plt.xlabel("Training samples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Train score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Test score")

    plt.legend(loc="best")
    return plt

## MLP Regressor

In [6]:
from sklearn.neural_network import MLPRegressor

def mLPRegressor(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    mlp = MLPRegressor(activation='logistic', solver='lbfgs')
    mlp.fit(X_train, y_train)
    predicts = mlp.predict(X_test)
    acc = mlp.score(X_test, y_test)
    mse = mean_squared_error(y_test, predicts)
    print("Accuracy: ", acc)
    print("MSE: ", mse)
    print("---------------------------------------")
    print(predicts.tolist())
    return mlp

In [14]:
X = data[['POTENCIA TERMICA BOMBA CALOR CARLOS', 'C_O_P BOMBA CALOR CARLOS']]
y = data['POTENCIA BOMBA CALOR CARLOS']
mlpCalorCarlos = mLPRegressor(X, y)

Accuracy:  0.9529316684094444
MSE:  90.694180390177
---------------------------------------
[2.15104580675432, 72.12735484800614, 109.32449603333579, 2.048977806646291, 59.04230281819335, 58.2827449833052, 33.59225330727173, 3.5059422909255593, 2.115379886769678, 118.97513179754093, 2.3523633496052945, 1.7861348100949188, 27.00034258756651, 83.16808734994123, 2.7407056387539566, 28.98303667543114, 69.21504473760693, 11.173393655587152, 2.34298742662821, 2.6481134002686364, 2.1153787385415814, 2.2568894398876695, 1.9232602515363704, -9.438791989580459, 1.9397056996650166, 3.376350542003472, 1.9015928163520677, 1.958272683001737, 2.3813796491602597, 2.1835245122861657, 9.665065211488736, 86.59404654486362, 92.47151701666742, 83.82501454163824, 57.26788506753519, 1.8790103774883606, 2.1387775994166383, 134.90020746929807, 18.124543282307428, 136.04922059228838, 116.6181262718669, 57.22699058425211, 1.7967230215722534, 1.9081555540722661, 89.97400553741741, 2.7407056387539566, 1.9313195306

## SVR

In [21]:
from sklearn.svm import SVR

def sVR(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    svr = SVR(gamma=0.4)
    svr.fit(X_train, y_train)
    predicts = svr.predict(X_test)
    acc = svr.score(X_test, y_test)
    mse = mean_squared_error(y_test, predicts)
    print("Accuracy: ", acc)
    print("MSE: ", mse)
    print("---------------------------------------")
    print(predicts.tolist())
    return svr

In [22]:
X = data[['POTENCIA TERMICA BOMBA CALOR CARLOS', 'C_O_P BOMBA CALOR CARLOS', 'TEMPERATURA EXTERIOR',
          'TEMPERATURA AMBIENTE BOMBA CALOR CARLOS', 'TEMPERATURA SALIDA BOMBA CALOR CARLOS']]
y = data['POTENCIA BOMBA CALOR CARLOS']
svrCalorCarlos = sVR(X, y)

Accuracy:  0.018929138005265633
MSE:  1857.815343346931
---------------------------------------
[4.954849396914789, 39.0252095831201, 10.4290007667332, 18.145111708323437, 0.5678000217138255, 10.58069715226671, 9.291889786556448, 0.39120359716818065, 32.2767919910693, 4.789098177805084, 0.456559447897261, 0.7852553292599271, 5.234414364941231, 0.5157530406455937, 0.3253281185486685, 8.4228831462657, 5.120250833197559, -0.008853086178412184, 19.40482007782184, 5.7573782271471865, 8.033423350829402, 1.3353139794019775, 30.280223452474047, 0.40585158248578956, 23.355440963642394, 10.967136623034829, 20.873421017048535, 14.928295617068684, 0.5103645514348578, 1.8282456542782572, 0.5861013007819391, 0.5605600590506512, 18.832894921096468, 24.592552943987293, 1.0206320468907375, 30.984615196509655, 13.886017807710218, 10.893579026387659, 13.160415671490016, 9.380091860229344, 1.1302626582523398, 6.605747416610891, 1.0948929050331984, 13.507498188041833, 12.972948722104174, 7.897153278983366,

## SGD Regressor

In [23]:
from sklearn.linear_model import SGDRegressor

def sGDRegressor(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    clf = SGDRegressor()
    clf.fit(X_train, y_train)
    predicts = clf.predict(X_test)
    acc = clf.score(X_test, y_test)
    mse = mean_squared_error(y_test, predicts)
    print("Accuracy: ", acc)
    print("MSE: ", mse)
    print("---------------------------------------")
    print(predicts.tolist())

In [24]:
X = data[['POTENCIA TERMICA BOMBA CALOR CARLOS', 'C_O_P BOMBA CALOR CARLOS', 'TEMPERATURA EXTERIOR',
          'TEMPERATURA AMBIENTE BOMBA CALOR CARLOS', 'TEMPERATURA SALIDA BOMBA CALOR CARLOS']]
y = data['POTENCIA BOMBA CALOR CARLOS']
svrCalorCarlos = sGDRegressor(X, y)

Accuracy:  -1.090958711505221e+22
MSE:  2.095602919326733e+25
---------------------------------------
[817587403114.5126, 4349301656832.331, 6670705835121.133, 7695383782174.718, 9730179220235.531, 1510505190464.4568, 3680819908567.9697, 1487464274992.1853, 3555089527443.656, 5760790243497.134, 8438899829383.961, 2045506542691.8462, 1706258334592.782, 6785396697108.144, 1541079831485.1504, 7614580896572.598, 2111647331078.4612, 1561186619120.343, 7445493828892.666, 1474950956187.2327, 1483142661705.9175, 3942564784822.3794, 5405273874330.21, 4777877468175.418, 4659605252635.401, 929852798779.7235, 4293792369693.3945, 6561105455328.765, 3965525875271.7354, 1931403363554.7793, 2141078565870.2131, 1678705507013.135, 6315664215335.158, 1686076387701.9185, 5481080686694.018, 1661597997059.8706, 6612884176023.981, 1747086226074.5535, 6376366834349.21, 1932738756208.1028, 1396281042858.4224, 2708580294086.7617, 5817540435566.843, 10109420790552.543, 3007161727731.9688, 7426282615948.576, 6824

## KNeighbors Regressor

In [33]:
from sklearn.neighbors import KNeighborsRegressor

def kNRegressor(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    neigh = KNeighborsRegressor(n_neighbors=10)
    neigh.fit(X_train, y_train) 
    predicts = neigh.predict(X_test)
    acc = neigh.score(X_test, y_test)
    mse = mean_squared_error(y_test, predicts)
    print("Accuracy: ", acc)
    print("MSE: ", mse)
    print("---------------------------------------")
    print(predicts.tolist())
    return neigh

In [34]:
X = data[['POTENCIA TERMICA BOMBA CALOR CARLOS', 'C_O_P BOMBA CALOR CARLOS', 'TEMPERATURA EXTERIOR',
          'TEMPERATURA AMBIENTE BOMBA CALOR CARLOS', 'TEMPERATURA SALIDA BOMBA CALOR CARLOS']]
y = data['POTENCIA BOMBA CALOR CARLOS']
knrCalorCarlos = kNRegressor(X, y)

Accuracy:  0.9672689593127789
MSE:  62.35012143454809
---------------------------------------
[0.5599999934434885, 76.59680252075195, 116.79946670532229, 19.513866609334954, 0.45000000298023196, 112.41813354492199, 4.395999839901925, 29.197333335876472, 77.82987060546877, 137.2682678222656, 58.990267944335926, 0.49333333373069765, 2.0231999695301064, 1.1490666896104809, 119.71279907226571, 0.4839999914169312, 39.37586669921875, 0.5119999945163729, 0.48999999463558197, 5.321999856829639, 129.7165351867676, 0.47999998927116405, 130.94186859130883, 0.4759999990463256, 69.00799942016602, 0.5, 0.46399999260902397, 104.14387054443375, 81.76213836669922, 0.5520000070333482, 70.19040298461914, 1.6400000214576687, 0.48199999034404756, 23.27946662902832, 1.2020000189542768, 0.5180000007152559, 78.91466903686525, 0.4979999989271164, 0.5, 0.4000000059604639, 0.5600000143051147, 89.93466567993165, 0.46399999260902397, 0.4886666625738144, 0.4600000023841856, 0.48199999034404756, 0.5159999936819075, 

## Linear Regression

In [21]:
from sklearn.linear_model import LinearRegression

def linearRegression(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    reg = LinearRegression()
    reg.fit(X_train, y_train)
    predicts = reg.predict(X_test)
    acc = reg.score(X_test, y_test)
    mse = mean_squared_error(y_test, predicts)
    print("Accuracy: ", acc)
    print("MSE: ", mse)
    print("---------------------------------------")
    print(predicts.tolist())
    return reg

In [22]:
X = data[['POTENCIA TERMICA BOMBA CALOR CARLOS', 'C_O_P BOMBA CALOR CARLOS', 'TEMPERATURA EXTERIOR',
          'TEMPERATURA AMBIENTE BOMBA CALOR CARLOS', 'TEMPERATURA SALIDA BOMBA CALOR CARLOS']]
y = data['POTENCIA BOMBA CALOR CARLOS']
regCalorCarlos = linearRegression(X, y)

Accuracy:  0.9443914785499744
MSE:  107.49062614278093
---------------------------------------
[70.0026998863145, 93.06812005311555, 61.8891964873325, -3.646156924859973, -2.9587321918859786, 45.40701366669524, 42.436071806834896, -5.348018828160889, 3.5208697424248356, 114.98762918015666, 1.0652737209817218, 7.468242396860489, 63.87568825869808, 12.80483326162507, -2.4592113805345264, 129.94134425808704, 32.30146355480453, -2.8952627998918743, -5.044317055900102, 106.02701061007298, 90.48087133961636, 1.3496271405436318, 0.8908053465267258, 1.2598401865729443, 26.18706129440605, -0.3310397444228599, 7.082889471667777, 0.2913029121231574, 88.78848284163894, 89.56590475653275, 49.343506767164264, 86.80797279685592, 55.504174911096584, -0.5466623250772855, -1.3288976448319545, 120.85107710498131, 36.084304927519035, 14.149846517316579, 56.07244668766907, 13.67943334408142, 1.068955658477691, 40.718006219415756, 0.26691180261034475, 63.51177925983796, 9.96897657510008, 5.741974818829149, 